In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
import sys
import matplotlib.pyplot as plt

sys.path.insert(0, "..")

In [33]:
# Cargar el DataFrame desde el archivo JSON
df = pd.read_json('../data/processed/scaled_data.json', orient='records', lines=True)

# Convertir las listas de coordenadas en arrays numpy
df['cop_x'] = df['cop_x'].apply(lambda x: np.array(x))
df['cop_y'] = df['cop_y'].apply(lambda x: np.array(x))

# Combinar las columnas de coordenadas en una sola entrada
X_cop_x = np.array(df['cop_x'].tolist())
X_cop_y = np.array(df['cop_y'].tolist())
X = np.stack((X_cop_x, X_cop_y), axis=-1)

# Convertir las etiquetas a números
label_map = {'Healthy': 0, 'Diabetic': 1, 'Neuropathic': 2}
y = df['class'].map(label_map).values

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_index, test_index = next(sss.split(X, y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

# Escalar los datos para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 2)).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, 2)).reshape(X_test.shape)

# Verificar las dimensiones de los conjuntos de datos
print("Shape de X_train:", X_train_scaled.shape)
print("Shape de X_test:", X_test_scaled.shape)
print("Shape de y_train:", y_train.shape)
print("Shape de y_test:", y_test.shape)


Shape de X_train: (3056, 500, 2)
Shape de X_test: (1311, 500, 2)
Shape de y_train: (3056,)
Shape de y_test: (1311,)


In [34]:
from keras.layers import Dropout

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(500, 2)))
model.add(Dropout(0.2))  # Dropout después de la primera capa LSTM
model.add(LSTM(units=50))
model.add(Dropout(0.2))  # Dropout después de la segunda capa LSTM
model.add(Dense(units=3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.save('LSTM.h5')


c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_40"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_78 (LSTM)                  │ (None, 500, 50)        │        10,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_66 (Dropout)            │ (None, 500, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_79 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_67 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 3)              │           153 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,953 (120.91 KB)

 Trainable params: 30,953 (120.91 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
# Evaluación del modelo en los datos de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss en datos de prueba: {loss:.4f}')
print(f'Accuracy en datos de prueba: {accuracy:.4f}')

Epoch 1/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 59s 530ms/step - accuracy: 0.3782 - loss: 1.0890 - val_accuracy: 0.4699 - val_loss: 1.0159
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 22s 229ms/step - accuracy: 0.4588 - loss: 1.0368 - val_accuracy: 0.4996 - val_loss: 1.0030
Epoch 3/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 22s 231ms/step - accuracy: 0.4693 - loss: 1.0225 - val_accuracy: 0.5187 - val_loss: 0.9915
Epoch 4/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 235ms/step - accuracy: 0.4816 - loss: 1.0154 - val_accuracy: 0.5408 - val_loss: 0.9791
Epoch 5/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 23s 237ms/step - accuracy: 0.5152 - loss: 0.9976 - val_accuracy: 0.4905 - val_loss: 1.0173
Epoch 6/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 22s 231ms/step - accuracy: 0.4944 - loss: 0.9907 - val_accuracy: 0.5294 - val_loss: 0.9655
Epoch 7/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 22s 229ms/step - accuracy: 0.5096 - loss: 0.9842 - val_accuracy: 0.5217 - val_loss: 0.9692
Epoch 8/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 22s 230ms/step - accuracy: 0.5089 - loss: 0.9715 - val_accu